In [55]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from data_preprocessing import Preprocess
from sklearn.model_selection import train_test_split

In [56]:
df = pd.read_csv('../../data/dialects_data.csv')

In [57]:
df['dialect'].unique()

array(['LY', 'MA', 'EG', 'LB', 'SD'], dtype=object)

In [58]:
preprocess=Preprocess(df)

cleaned_df = preprocess.preprocess()

In [59]:
cleaned_df.head()

,id,text,dialect
0,1009754958479151232,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس...,LY
1,1009794751548313600,الليبيين متقلبين بس بالنسبة ليا انا ميليشياوي ...,LY
2,1019989115490787200,كل تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ ان...,LY
3,1035479791758135168,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل...,LY
4,1035481122921164800,شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتي...,LY


In [60]:
labels = cleaned_df['dialect'].unique().tolist()
labels

['LY', 'MA', 'EG', 'LB', 'SD']

In [61]:
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

In [62]:
id2label, label2id

({0: 'LY', 1: 'MA', 2: 'EG', 3: 'LB', 4: 'SD'},
 {'LY': 0, 'MA': 1, 'EG': 2, 'LB': 3, 'SD': 4})

In [63]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = list(examples["text"].values())
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [64]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [65]:
one=OneHotEncoder()


In [66]:
#create a column for each dialect
one.fit(cleaned_df[['dialect']])
dialects=one.transform(cleaned_df[['dialect']]).toarray()
dialects

c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spa

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [67]:
cleaned_df[labels] = dialects
cleaned_df.head()

,id,text,dialect,LY,MA,EG,LB,SD
0,1009754958479151232,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس...,LY,0.0,0.0,1.0,0.0,0.0
1,1009794751548313600,الليبيين متقلبين بس بالنسبة ليا انا ميليشياوي ...,LY,0.0,0.0,1.0,0.0,0.0
2,1019989115490787200,كل تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ ان...,LY,0.0,0.0,1.0,0.0,0.0
3,1035479791758135168,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل...,LY,0.0,0.0,1.0,0.0,0.0
4,1035481122921164800,شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتي...,LY,0.0,0.0,1.0,0.0,0.0


In [68]:

from datasets import Dataset, DatasetDict

In [69]:
cleaned_df.head()

,id,text,dialect,LY,MA,EG,LB,SD
0,1009754958479151232,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس...,LY,0.0,0.0,1.0,0.0,0.0
1,1009794751548313600,الليبيين متقلبين بس بالنسبة ليا انا ميليشياوي ...,LY,0.0,0.0,1.0,0.0,0.0
2,1019989115490787200,كل تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ ان...,LY,0.0,0.0,1.0,0.0,0.0
3,1035479791758135168,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل...,LY,0.0,0.0,1.0,0.0,0.0
4,1035481122921164800,شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتي...,LY,0.0,0.0,1.0,0.0,0.0


In [70]:
x_train, x_test = train_test_split(cleaned_df.drop("id",axis=1), test_size=0.2, random_state=42,stratify=cleaned_df['dialect'])

c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [71]:
train_dataset = Dataset.from_pandas(x_train)
test_dataset = Dataset.from_pandas(x_test)

In [72]:
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

In [73]:
dataset_dict['train'][0]

{'text': 'والله بالعكس جو سمح لكن صقع سم لين خلاص',
 'dialect': 'LY',
 'LY': 0.0,
 'MA': 0.0,
 'EG': 1.0,
 'LB': 0.0,
 'SD': 0.0,
 '__index_level_0__': 30200}

In [74]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [79]:
encoded_dataset = dataset_dict.map(preprocess_data, batched=True, remove_columns=dataset_dict['train'].column_names)

Map:   0%|          | 0/118180 [00:00<?, ? examples/s]

Map:   0%|          | 0/29545 [00:00<?, ? examples/s]

In [80]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [81]:
tokenizer.decode(example['input_ids'])

'[CLS] والله بالعكس جو سمح لكن صقع سم لين خلاص [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [82]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['EG']

In [83]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [84]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
batch_size = 8
metric_name = "f1"

In [86]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

c:\Users\zacks\anaconda3\envs\DataTraining\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [87]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [88]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [89]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1298, 25573, 23673, 23673, 14157,  1271, 25573, 23673, 29830,
        29835, 29824,  1275, 29836,  1282, 22192, 29820,  1294, 29835, 15915,
         1284, 29834, 29830,  1282, 22192,  1294, 14498, 15915,  1277, 23673,
        25573, 29826,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [90]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.6631, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1042, -0.4823,  0.0990,  0.2309,  0.0768]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [93]:
#f1 score
from datasets import load_metric

In [97]:
metric = load_metric("f1", trust_remote_code=True)

In [98]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=metric
)

In [100]:
import torch

In [101]:
#move the model to the GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
device

'cpu'

In [99]:
trainer.train()

  0%|          | 0/73865 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()